In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision  # modules and transforms for computer vision
from matplotlib import pyplot as plt
from tqdm.auto import tqdm  # progress bar
import os
from glob import glob
import nibabel as nib
from torch.utils.data.dataset import Dataset
import random
import pathlib
import os
import math
from torch.utils.tensorboard import SummaryWriter
from skimage.metrics import structural_similarity as ssim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/Shareddrives/Deep Learning Project/Pix2Pix/results/'

In [ ]:
# MAE, MSE, SSIM, PSNR

def mse(input,targets):
  mse_loss = np.mean(np.square(np.abs(input - targets)))
  return mse_loss

def mae(input,targets):
  mae_loss = np.mean(np.abs(input - targets))
  return mae_loss

# def ssim(input,targets):
#   s_loss = ssim(input,targets)
#   return s_loss
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio
def ssim(synthetic_img,reference_img):
  s_loss = structural_similarity(reference_img, synthetic_img)
  #psnr_val = peak_signal_noise_ratio(reference_img, synthetic_img, data_range=reference_img.max() - reference_img.min())

def psnr(input,targets):
  max_pixel = 1
  p_loss = 20*np.log10(max_pixel / np.sqrt(mse(input,targets)))
  return p_loss

In [ ]:
import imageio.v3 as iio

In [ ]:
#Calculate Metrics on 3D Images
mse_list=[]
mae_list=[]
ssim_list=[]
psnr_list=[]

mse_sub_list=[]
mae_sub_list=[]
ssim_sub_list=[]
psnr_sub_list=[]

patient=['16','17','18']
for i in range(3): #Patient 16,17,18
  #pat_ind= sorted(os.listdir(os.path.join(path, patient[i])))
  for j in range (40): #Loop over all 40 slices

    pred=path+patient[i]+'/'+patient[i]+"__"+str(j+101)+"_fake_B.png"
    gt=path+patient[i]+'/'+patient[i]+"__"+str(j+101)+"_real_B.png"
    
    test_image=iio.imread(pred)
    test_gt=iio.imread(gt)

    test_image = test_image/255
    test_gt = test_gt/255


    mse_loss=mse(test_image,test_gt)
    mae_loss=mae(test_image,test_gt)
    s_loss=structural_similarity(test_gt,test_image,channel_axis=2)
    p_loss=psnr(test_image,test_gt)
    mse_sub_list.append(mse_loss)
    mae_sub_list.append(mae_loss)
    ssim_sub_list.append(s_loss)
    psnr_sub_list.append(p_loss)

  mse_list.append(np.mean(mse_sub_list))
  mae_list.append(np.mean(mae_sub_list))
  ssim_list.append(np.mean(ssim_sub_list))
  psnr_list.append(np.mean(psnr_sub_list))

avg_mse=np.mean(mse_list)
avg_mae=np.mean(mae_list)
avg_ssim=np.mean(ssim_list)
avg_psnr=np.mean(psnr_list)
mse_list.append(avg_mse)
mae_list.append(avg_mae)
ssim_list.append(avg_ssim)
psnr_list.append(avg_psnr)

In [ ]:
data={'MSE':mse_list,'MAE':mae_list,"PSNR":psnr_list,"SSIM":ssim_list}
df=pd.DataFrame(data, index=['Patient 1','Patient 2','Patient 3','Average'])
#df=pd.DataFrame(data, index=['Patient 1','Patient 2','Average'])
display(df)

,MSE,MAE,PSNR,SSIM
Patient 1,0.003990,0.031122,24.032352,0.873904
Patient 2,0.003837,0.030754,24.204060,0.873121
Patient 3,0.003697,0.029829,24.377309,0.876473
Average,0.003842,0.030568,24.204573,0.874499


In [ ]:
print(np.min(test_gt))

0
